<img src="img/swarm-logo.png">
<small>https://medium.com/southbridge/%D0%BE%D0%B4%D0%BD%D0%BE%D1%80%D0%B0%D0%B7%D0%BE%D0%B2%D1%8B%D0%B5-%D0%BA%D0%BE%D0%BD%D1%82%D0%B5%D0%B9%D0%BD%D0%B5%D1%80%D1%8B-%D0%B2-docker-swarm-a14a60117dec</small>

Docker Swarm - это система оркестрации множества контейнеров на множестве машин. В ее обязаности входит слежение за состоянием контейнеров, перемещение их между машинами, обеспечение сетевой связанности между машинами, предоставление пользователю единого интерфейса для работы с кластером и другие технические задачи по поддержанию работоспособности кластера.

Как и в предыдущей лабораторной, мы будем использовать сервис Play With Docker для демонстрации возможностей этого инструмента.

# [Открыть Play With Docker](https://labs.play-with-docker.com/)

# Объединение в кластер

Чтобы начать управлять кластером, вначале необходимо объединить разрозненные машины в единую систему. Для этого создадим сразу 3 виртуальные машины. В Play With Docker они будут называться node1, node2 и node3 и каждая их них будет иметь свой IP адрес.

В Docker Swarm есть два типа машин - менеджеры и рабочие. Менеджеры занимаются управлением всеми машинами. Если потребуется производить какие-либо манипуляции с кластером, это потребуется делать через машину-менеджера. 
Машины-рабочие не делают практически ничего, кроме как слушают команды от машин-менеджеров и беспрекословно выполняют их команды.

Для наших целей назначим node1 машиной-менеджером, а node2 и node3 - машинами-рабочими.

Для этого на машине node1 запустим команду, которая инициализирует ее как менеджера. В этой команде необходимо будет указать IP адрес этой машины, через который она будет общаться с другими машинами кластера. В моем случае это 192.168.0.13 (в вашем случае IP может отличатся, внимательно посмотрите на то, какие IP выдал вам play with docker).

```bash
docker swarm init --advertise-addr 192.168.0.13
```

Отлично! Теперь к этому менеджеру необходимо подключить оставшиеся машины в качестве рабочих. После предыдущей команды на экране должна появится команда для подключения. Если вы ее уже потеряли, то можно руками еще раз спросить, какой токен требуется для подключения рабочих - для этого запустим следующую команду.

```bash
docker swarm join-token -q worker
```

Формируем токен, переключаемся на node2 и запускаем команду на подключение. Для меня команда выглядит следующим образом:
```bash
docker swarm join --token SWMTKN-1-3kvfsmba4zi7zxwcixwf4obzlqyd1jyrhx9xt365lzbf7nzwp8-e0fyc576f5c7x7x9mwr960lmy 192.168.0.13:2377
```

Важно, что токен может отличатся, а также адрес `192.168.0.13` (адрес менеджера к которому подключаемся) также может отличаться - вставьте туда актуальный адрес node1.

Если все сделано правильно, то машина должна была подключиться к менеджеру в качестве рабочего.

Точно такую же операцию необходимо произвести для машины node3.

Поздравляю! Мы только что создали кластер!



# Создаем первый сервис

Создавать сервисы в Docker Swarm можно просто из командной строки. Для этого есть команда `docker service create`.

Запустим сервис по отображению состяния кластера, чтобы мы могли визуально следить за происходящим.

```bash
docker service create \
  --name=viz \
  --publish=8080:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  dockersamples/visualizer
```

Некоторые параметры запуска здесь выходят за рамки нашего курса, но все же дадим краткое описание того, что мы запускаем.
* `--name` дает название для этого сервиса. Может быть любой строкой
* `--publish=8080:8080/tcp` это уже знакомый нам проброс портов - все внешние соеднинения с 8080 мы отправляем на внутренний 8080
* `--constraint=node.role==manager` говорит о том, что этот сервис можно запускать только на машинах-менеджерах. Это необходимо, так как только менеджеры знают информацию для отображения
* `--mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock` это уже знакомый проброс файлов и директорий внутрь контейнера. Визуализатору требуется доступ до докера, но так как сам он по себе находится внутри контейнера, нужно в явном виде предоставить ему доступ до докера в основной ОС
* `dockersamples/visualizer` название докер-контейнера для запуска

Запустив это команду на node1 (это важно, так как это машина-менеджер), в кластере развернется этот контейнер. Посмотреть состояние запущенных сервиов в кластере можно с помощью команды 

```bash
docker service ls
```

Там должен отображаться наш новый сервис.

Попробуем к нему подключится, открыв порт 8080 - должна отобразиться схема с отобржением трех машин и контейнеров на них.

# Особенности запуска на кластере

### Общая сеть для связи между контейнерами

По умолчанию общая сеть есть только у контейнеров на одной машине. Docker Swarm позволяет сделать общую сеть между контейнерами сразу на всем кластере. Тогда вне зависимости от того, где физически запущен сервис, он всегда сможет достучаться до любого другого контейнера. 

Создадим общую сеть на всем кластере и назовем ее backnet.
```bash
docker network create --driver overlay backnet
```

### Конфигурирование 

Монтирование файлов и директорий также работает только в рамках одной машины. Это означает, что если бы мы захотели запустить сервис, который требует внешней конфигурации (например nginx из предыдущей лабораторной), то пришлось бы положить руками этот файл конфигурации на все машины. К счастью, в docker swarm есть специальный механизм распределенного конфигрурирования. Он позволяет указать файл конфигурации и docker swarm самостоятельно передаст его на необходимые машины и подключит к нужным контейнерам.


### Сборка конейтнера

Сборка самого контейнера также происходит на одной машине. Это означает, что на других машинах просто не будет этого контейнера, чтобы запустить. Для того, чтобы решить эту проблему, мы можем собрать и выложить его в реестр образов. Для наших целей воспользуемся реестром от создателей Docker - Docker Hub. Этот сервис позволяет размещать у себя собранные пользователями контейнеры. 

Соберем и выложим наш контейнер с файловым сервисом.


**file-server.sh**
```bash
export DIR=$(cat /etc/file-config.txt)

mkdir -p $DIR

python3 -m http.server --directory "$DIR" --bind 0.0.0.0 8080
```

**Dockerfile**
```
FROM python:3.7

COPY file-server.sh file-server.sh

ENTRYPOINT ["bash", "file-server.sh"]
```

Теперь соберем наш образ

```bash
docker build -t file-server:latest .
```

Осталось его выложить в Docker Hub. Для начала создадим публичный репозиторий - https://hub.docker.com/repository/create . Назовем его также - `file-server`. Каждый образ прикрепляется к именно автора. Так как мой никнейм на Docker Hub `adkosmos`, то полное название моего контейнера будет называться `adkosmos/file-server:latest`. В вашем случае необходимо будет использовать **ваш** никнейм.

Авторизируемся на Docker Hub
```bash
docker login
```

Выложим собранный нами образ.

```bash
docker tag file-server:latest adkosmos/file-server:latest
docker push adkosmos/file-server:latest
```

Поздравляю, вы только что выложили свой первый Docker образ!


# Запуск на кластере

Попробуем запустить наши сервисы, которые мы разрабатывали в предыдущей лабораторной. Для этого придется лишь немного модифицировать уже написанный docker-compose.yaml - необходимо подключить сервисы к созданной нами общей сети и добавить конфигурирование через docker swarm.

**config.txt**
```
/shared/folder
```

**configuration.nginx**
```
user root;
worker_processes  4;

error_log  /var/log/nginx/error.log warn;
pid        /var/run/nginx.pid;

events {
    worker_connections  4096;
}

http {
    include       /etc/nginx/mime.types;
    default_type  application/octet-stream;
    log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
                      '$status $body_bytes_sent "$http_referer" '
                      '"$http_user_agent" "$http_x_forwarded_for"';

    access_log  /var/log/nginx/access.log  main;

    sendfile            on;
    tcp_nopush          on;
    keepalive_timeout   65;

    server {
        listen  8888;

        location /file-server-1 {
            proxy_pass http://file-server-1:8080;   # Перенаправляем в соседний контейнер по именно контейнера!
            rewrite ^/file-server-1/(.*) /$1 break;  # Удаляем префикс /web-server-1
        }
        
        location /file-server-2 {
            proxy_pass http://file-server-2:8080;
            rewrite ^/file-server-2/(.*) /$1 break;
        }
    }
}
```

Все эти файлы просто скопированы и никак не изменялись. Осталось создать docker-compose.yaml.

**docker-compose.yaml**
```yaml
version: "3.7"

services:
  file-server-1:
    image: adkosmos/file-server:latest  # Образ, который мы только что собрали и выложили
    volumes:
      - /machine-info:/shared/folder # Текущая директория
    configs:
      - source: file-server-config
        target: /etc/file-config.txt
  file-server-2:
    image: adkosmos/file-server:latest  # Образ, который мы только что собрали и выложили
    volumes:
      - /etc:/shared/folder
    configs:
      - source: file-server-config
        target: /etc/file-config.txt
  proxy:
    image: nginx:1.17
    ports:
      - 8888:8888
    configs:
      - source: nginx-config
        target: /etc/nginx/nginx.conf  # Подключаем конфигурацию
      
networks:
  backnet:
    external: true  # Используем созданную извне сеть с названием backnet

configs:  # Задаем файлы конфигураций, которые необходимо распределить по кластеру
  file-server-config:
    file: ./config.txt
  nginx-config:
    file: ./configuration.nginx 
```

Чтобы мы могли видеть, на какой машине работает сервис, к которому мы подключились, создадим директорию /machine-info на каждой машине и создадим в них файлы с названиями машин (то есть node1, node2, node3). Эту директорию подключим к file-server-1.


*на всех машинах*
```bash
mkdir -p /machine-info
touch /machine-info/node-{N}
```

Настало время запускать наши сервисы. Назовем наш stack file-service.

*на node1 (машине-менежеру)*
```bash
docker stack deploy --compose-file docker-compose.yaml file-service
```

Можешь проверить состояние с помощью команды
```bash
docker service ls
```

Через некоторое время все контейнеры должны запуститься!

Чтобы посмотреть, как Docker Swarm раскидал конейнеры по машинам, откроем еще раз 8080 порт, на котором все еще работает визуализатор.

# Проверяем работу

Откроем `8888` порт на node1 и там же добавим `/file-server-1/`. Можно видеть директорию с нашего файлового сервиса. 

Здесь важно отметить, что не важно, находятся nginx и file-server-1 на одной машине или нет. Благодаря общей внутренней сети кластера, они без проблем общаются.

Так же стоит отметить, что не важно, к какой именно машине подключаться. Попробуем открыть `8888` порт на node2 и на node3 - результат будет точно такой же. Это происходит благодаря обобщенному интерфейса для пользователя. Запрос к любой машине будет перенаправлен в нужный контейнер.

# Масштабируем наш сервис. 

Давайте удвоим каждый наш контейнер. Для этого необходимо указать параметр репликации в нашем `docker-compose.yaml`

**docker-compose.yaml**
```yaml
version: "3.7"

services:
  file-server-1:
    image: adkosmos/file-server:latest  # Образ, который мы только что собрали и выложили
    deploy:
      mode: replicated
      replicas: 2  # Добавляем 2 реплики этого сервиса
    volumes:
      - /machine-info:/shared/folder # Текущая директория
    configs:
      - source: file-server-config
        target: /etc/file-config.txt
  file-server-2:
    image: adkosmos/file-server:latest  # Образ, который мы только что собрали и выложили
    deploy:
      mode: replicated
      replicas: 2  # Добавляем 2 реплики этого сервиса
    volumes:
      - /etc:/shared/folder
    configs:
      - source: file-server-config
        target: /etc/file-config.txt
  proxy:
    image: nginx:1.17
    deploy:
      mode: replicated
      replicas: 2  # Добавляем 2 реплики этого сервиса
    ports:
      - 8888:8888
    configs:
      - source: nginx-config
        target: /etc/nginx/nginx.conf  # Подключаем конфигурацию

networks:
  backnet:
    external: true  # Используем созданную извне сеть с названием backnet

configs:  # Задаем файлы конфигураций, которые необходимо распределить по кластеру
  file-server-config:
    file: ./config.txt
  nginx-config:
    file: ./configuration.nginx 
```

Перезапускаем наши сервисы

```
docker stack deploy --compose-file docker-compose.yaml file-service
```

Проверяем. `service ls` должен показать, что теперь у сервиса по две реплики
```
docker service ls
```

Открываем `8080` и смотрим, как теперь контейнеры распределены по кластеру.

Попробуем теперь открыть `8888`. Так как теперь у нас два прокси-сервера, то кластер переадресует нас к любому из них. Более того, так как файловых серверов теперь тоже по два, прокси-сервер также будет перенаправлять нас к случайному их сервисов.

Для того, чтобы это увидеть, пообновляйте путь `/file-server-1/` несколько раз. Если контейнеры физически запустились на разных машинах, то мы будем видеть разные файлы при обновлении.

Вся схема работы нашей системы выглядит следующим образом:

<img src="img/docker-swarm-file-service-scheme.png">